# Предобработка данных и функции потерь в линейной регрессии

## Данные
Для демонстраций загрузим набор данных [Automobile Data Set](https://archive.ics.uci.edu/ml/datasets/Automobile). В данных присутствуют категориальные, целочисленные и вещественнозначные признаки.

In [ ]:
import pandas as pd
X_raw = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data", \
                    header=None, na_values=["?"])

In [ ]:
X_raw.head()

In [ ]:
y = X_raw[25]
X_raw = X_raw.drop(25, axis=1)

## Предобработка данных

Предобработка данных важна при применении любых методов машинного обучения, а в особенности для линейных моделей. В sklearn предобработку удобно делать с помощью различных модулей [preprocessing](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) или методов библиотеки pandas.

### Заполнение пропусков
В матрице объекты-признаки могут быть пропущенные значения, и это вызовет ошибку при попытке передать такую матрицу в функцию обучения модели или даже предобработки. Если пропусков немного, можно удалить объекты с пропусками из обучающей выборки. Заполнить пропуски можно разными способами:
* заполнить средними (mean, median);
* предсказывать пропущенные значения по непропущенным.

Часто используют первый вариант - он проще. Для заполнения константами можно использовать метод датафрейма `fillna`, для замены средними - класс `impute.SimpleImputer` (в более старых версиях `scikit-learn` - `preprocessing.Imputer`).

In [ ]:
X_raw.isnull().any().any()

In [ ]:
X_raw.isnull().sum()

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
# для удобства работы с нашим датасетом создаем маску, указывающую на столбцы с категориальными признаками
cat_features_mask = (X_raw.dtypes == "object").values # категориальные признаки имеют тип "object"

In [ ]:
# для вещественнозначных признаков заполним пропуски средними
X_real = X_raw[X_raw.columns[~cat_features_mask]]
mis_replacer = SimpleImputer(strategy="mean")
X_no_mis_real = pd.DataFrame(data=mis_replacer.fit_transform(X_real), columns=X_real.columns)

In [ ]:
# для категориальных - пустыми строками
X_cat = X_raw[X_raw.columns[cat_features_mask]].fillna("")
X_no_mis = pd.concat([X_no_mis_real, X_cat], axis=1)

In [ ]:
X_no_mis.head()

In [ ]:
X_no_mis.isnull().any().any()

### Преобразование нечисловых признаков
Как вы помните, большниство моделей машинного обучения требуют, чтобы на вход функции обучения подавалас матрица состоящая из вещественных чисел. В процессе обучения используются свойства вещественных чисел, в частности, возможность сравнения и применения арифметических операций. Поэтому, даже если формально в матрице объекты-признаки записаны числовые значения, нужно всегда анализировать, можно ли относиться к ним как к числам. 

__Пример:__ некоторые признаки могут задаваться целочисленными хешами или id (например, id пользователя соц. сети), однако нельзя сложить двух пользователей и получить третьего, исходя из их id (как это может сделать линейная модель).

Напоминаем, что для обработки категориальных признаков, часто применяют [one-hot encoding](http://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features) в результате чего, вместо одного признака добавится $K$ бинарных признаков - по одному на каждое возможное значение исходного признака. В `sklearn` это можно сделать с помощью классов `OneHotEncoder`, но проще использовать функцию `pd.get_dummies`.

Следует заметить, что в новой матрице будет очень много нулевых значений. Чтобы не хранить их в памяти, можно задать параметр `OneHotEncoder(sparse=True)` или `pd.get_dummies(sparse=True)`, и метод вернет [разреженную матрицу](http://docs.scipy.org/doc/scipy/reference/sparse.html), в которой хранятся только ненулевые значения. Выполнение некоторых операций с такой матрицей может быть неэффективным, однако большинство методов sklearn умеют работать с разреженными матрицами.

__Вопрос:__ стоит ли применять one-hot encoding для признаков с большим числом категорий (например, id)? Почему?

__Вопрос:__ какая проблема возникнет при применении вышеописанного способа кодирования для обучения линейной регрессии?
    
Необходимо удалить один из столбцов, созданных для каждого признака. Для этого в get_dummies надо поставить drop_first=True.

In [ ]:
X_no_mis.shape

In [ ]:
X_dum = pd.get_dummies(X_no_mis, drop_first=True)
print(X_dum.shape)
X_dum.head()

Помимо категориальных, преобразования требуют, например, строковые признаки. Их можно превращать в матрицу частот слов [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer), матрицу частот буквосочетаний фиксированной длины, можно извлекать другие признаки (например, длина строки).

### Масштабирование признаков
При начале работы с данными всегда рекомендуется приводить все признаки к одному масштабу.  Это важно по нескольким причинам:
* ускорение обучения модели;
* улучшение численной устойчивости при работе с матрицей объекты-признаки
* для линейных моделей: интерпретация весов при признаках как меры их значимости

[Полезная ссылка](https://towardsdatascience.com/understand-data-normalization-in-machine-learning-8ff3062101f0)

Данные можно масштабировать по-разному:

- Стандартизация - вычитание среднего из каждого признака и деление на стандартное отклонение (`StandardScaler` в `sklearn`).

- Нормализация - вычитание минимума из каждого признака, а затем деление на разницу максимального и минимального значения (`MinMaxScaler` в `sklearn`).

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
normalizer = MinMaxScaler()
X_real_norm_np = normalizer.fit_transform(X_dum)
X = pd.DataFrame(data=X_real_norm_np)

In [ ]:
X.head()

### Добавление признаков
Особенно важным моментом для линейной регрессии является нелинейное преобразование признаков. Это позволяет использовать линейную регрессию для моделирования нелинейных зависимостей. 

Наиболее популярны следующие преобразования: 
- полиномиальные признаки (PolynomialFeatures в sklearn)
- взятие логарифма
- квадратного корня
- применение тригонометрических функий.

Например, посмотрев на данные, мы можем заметить, что зависимость целевой переменной от шестого признака скорее квадратичная, чем линейная:

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
plt.scatter(X[6], y)
plt.show()

In [ ]:
plt.scatter(X[6]**2, y)
plt.show()

> А для признака номер 13 линеаризовать зависимость получается с помощью функции $\frac 1 {\sqrt{x}}$

In [ ]:
plt.scatter(X[13], y)
plt.show()

In [ ]:
plt.scatter(1 / np.sqrt(X[13]), y)
plt.show()

__Обратите внимание__: при генерации полиномиальных признаков матрица объекты-признаки может занимать много памяти.

## Функции потерь в регрессии

Функционал качества в задачах обучения с учителем обычно задается в виде суммы по объектам выборки:
$$Q(a) = \frac 1 \ell \sum_{i=1}^\ell L(y_i, a(x_i)),$$
где $L(\cdot, \cdot)$ - функция потерь, задающая штраф за разницу между предсказанием и истинным значением целевого признака. Свойства функции потерь:
* $L(y_i, a(x_i)) \geqslant 0$;
* $L(y_i, y_i) = 0$. 

Функционал качества должен в первую очередь отвечать требованиям заказчика, при этом математические свойства функции потерь могут быть неудобны для оптимизации. 

__Пример:__ если мы не различаем маленькие ошибки (между 0.01 и 0.1 нет особой разницы), но зато не хотим получать большие ошибки, можно использовать следующую функцию потерь:

$$L(y_i, a(x_i)) = [| y_i - a(x_i) | < \varepsilon],$$ $\varepsilon$ - допустимая разница между предсказанием и фактом.


### Среднеквадратичная и средняя абсолютная ошибка
Кроме требований заказчика, функционал качества должен учитывать математические особенности модели - например, устойчивость к шумовым объектам. 

В линейной регрессии Mean Squared Error: $L(y_i, a(x_i)) = (a(x_i) - y_i)^2$ не обладает этим свойством, потому что задает очень большие штрафы за большие отклонения от фактического значения. 

$$MSE (a, X, Y) = \sum^L_{i=1}(a(x_i) - y_i)^2$$

Рассмотрим это явление на примере. Выберем один признак, от которого целевой признак (имеющий индекс 15 в матрице X) зависит практически линейно. Добавим к выборке два объекта-выброса и посмотрим, как изменится оптимизированная на MSE прямая.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
X_subset = X[[7, 15]].values
X_subset_modified = np.vstack((X_subset, [[1, 90], [2, 50]])) # добавление двух шумовых точек

In [ ]:
def scatter_points_and_plot_line_MSE(X_subset):
    plt.scatter(X_subset[:, 0], X_subset[:, 1])   # визуализируем точки
    lr = LinearRegression()
    lr.fit(X_subset[:, 0][:, np.newaxis], X_subset[:, 1])  # вычислим веса линейной модели
    grid = np.linspace(0, 2, 100)
    line = lr.predict(grid[:, np.newaxis])
    plt.plot(grid, line)   # визуализируем прямую

In [ ]:
plt.figure(figsize=(20, 5))
plt.subplot(1, 2, 1)
scatter_points_and_plot_line_MSE(X_subset)
plt.ylim(-20, 100)
plt.xlabel("x")
plt.ylabel("y")
plt.subplot(1, 2, 2)
scatter_points_and_plot_line_MSE(X_subset_modified)
plt.ylim(-20, 100)
plt.xlabel("x")
plt.show()

**Задание:** Реализуйте функцию для подсчета MSE с использованием numpy

In [ ]:
def MSE(y: np.array, y_pred: np.array):
    return # YOUR CODE HERE


a = np.array([11,20,19,17,10])
pred = np.array([12,18,19.5,18,9])
mse = MSE(y=a, y_pred=pred)
print("The Mean Square Error is: " , mse)
assert mse == 1.45 

Среднеквадратичная ошибка подходит для сравнения двух моделей или для контроля качества во время обучения. Из-за того, разница возводится в квадрат - сложно дать этому числу интерпретацию. Для лучшей интерпретации используется Root Mean Square Error (RMSE). Таким образом, если MSE показывает разницу в квадратных единицах, значение RMSE сохранится в исходных. 

$$RMSE (a, X, Y) = \sqrt{MSE (a, X)} = \sqrt{ \sum^L_{i=1}(a(x_i) - y_i)^2}$$

**Задание:** Реализуйте функцию для подсчета RMSE с использованием numpy

In [ ]:
def RMSE(y, y_pred):
    return # YOUR CODE HERE


rmse = RMSE(y=a, y_pred=pred)
print("The Root Mean Square Error is: " , rmse)
assert rmse == 1.2041594578792296

Коэффициент детерминации $R^2$ показывает долю дисперсии в целевой переменной, которая обьяснена зависимыми переменными. $R^2$ можно интерпретировать как некоторого рода нормированное MSE.

$$R^2(a, X, Y) = 1 - \frac {\sum^L_{i=1}(a(x_i) - y_i)^2}{\sum^L_{i=1}(y_i - \bar{y})^2}$$

**Задание:** реализуйте функцию для вычисления $R^2$

In [ ]:
def R_squared(y, y_pred):
    return # YOUR CODE HERE

r_squared = R_squared(y=a, y_pred=pred)
print('The R2 is:', r_squared)

Из-за того, что мы учитываем квадрат отклонения - шумовые объекты могут сильно изменить наклон прямой. Поэтому в качестве альтернативы MSE можно использовать Mean Absolute Error: $L(y_i, a(x_i)) = |a(x_i) - y_i|$.

$$MAE(a, X, Y) = \frac {1}{L} \sum^L_{i=1}|a(x_i) - y_i|$$

Теперь обучим регрессию, оптимизируя MAE. В `sklearn` такая регрессия не реализована, но можно использовать модуль `statsmodels` - более формально, необходимая модель может быть получена с помощью обучения квантильной регрессии с параметром `q=0.5`.

**Задание:** Реализуйте функцию для подсчета MAE с использованием numpy

In [ ]:
def MAE(y, y_pred):
    return # YOUR CODE HERE


mae = MAE(y=a, y_pred=pred)
print("The Mean Absolute Error is: " , mae)
assert mae == 1.1

!pip install --user git+https://github.com/statsmodels/statsmodels # (если не работает импортирование библиотек ниже)

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
def scatter_points_and_plot_line_MAE(X_subset):
    # задаем зависимость переменной f15 от переменной f7 и передаем данные
    mod = smf.quantreg('f15 ~ f7', pd.DataFrame(data=X_subset, columns=["f7", "f15"]))
    res = mod.fit(q=0.5)
    
    # визуализируем точки
    plt.scatter(X_subset[:, 0], X_subset[:, 1])
    grid = np.linspace(0, 2, 100)
    
    # визуализируем прямую
    plt.plot(grid, grid * res.params["f7"] + res.params["Intercept"])
    return mod, res

In [ ]:
plt.figure(figsize=(20, 5))
plt.subplot(1, 2, 1)
model, result = scatter_points_and_plot_line_MAE(X_subset)
plt.ylim(-20, 100)
plt.xlabel("x")
plt.ylabel("y")
plt.subplot(1, 2, 2)
model, result = scatter_points_and_plot_line_MAE(X_subset_modified)
plt.ylim(-20, 100)
plt.xlabel("x")

Прямая практически не изменила направление из-за выбросов.

Попробуем добавить больше шумовых объектов:

In [ ]:
np.random.seed(13)
X_subset_modified_twice = np.vstack((X_subset_modified, np.random.randint(5, size=60).reshape(-1, 2)*[1, 30])) # добавление 30 шумовых точек

In [ ]:
plt.figure(figsize=(20, 5))
plt.subplot(1, 2, 1)
model, result = scatter_points_and_plot_line_MAE(X_subset)
plt.ylim(-20, 100)
plt.xlabel("x")
plt.ylabel("y")
plt.subplot(1, 2, 2)
model, result = scatter_points_and_plot_line_MAE(X_subset_modified_twice)
plt.ylim(-20, 100)
plt.xlabel("x")

Под таким количеством выбросов, изменилась даже регрессия над MAE

---

Рассмотрим некоторые свойства MSE и MAE.

Допустим алгоритм возвращает константное предсказание: $a(x) = C, C \in R$. В качестве примера такого алгоритма можно представить предсказание прибыли в январе константой, равной средней прибыли за январь всех предыдущих лет работы.

__Задача.__ Найдите $C$, минимизирующий среднеквадратичную ошибку.

__Задача.__ Найдите $C$, минимизирующий среднюю абсолютную ошибку.

Поскольку средняя абсолютная ошибка не является дифференцируемой по $w$ функцией, оптимизировать ее напрямую методом градиентного спуска не удастся. Для этого используются [субградиентные](https://ru.wikipedia.org/wiki/Субдифференциал) (что в почти не меняет процедуры спуска, только вместо градиента берется один из субградиентов) или другие методы.

### Huber Loss
Иногда используют "гибрид" MAE и MSE, который, как и MAE, устойчив к шумовым объектам, и как и MSE, мало штрафует малые отклонения от фактического значения целевого признака - Huber Loss:
$$L_i(y_i, a(x_i)) = \phi_\varepsilon(a(x_i) - y_i)$$
$$\phi_\varepsilon(z) = \begin{cases} \frac 1 2 z^2, - \varepsilon < z < \varepsilon, \\\varepsilon (|z| - \frac 1 2 \varepsilon), иначе \\ \end{cases}$$

Можно проверить, что у этой функции существует непрерывная первая проиводная во всех точках.

Оптимизация Huber Loss реализована в sklearn:

In [ ]:
from sklearn.linear_model import HuberRegressor

**Задание:** реализуйте функцию потерь Хьюбера

In [ ]:
def Huber(y, y_pred):
    #YOUR CODE HERE
    pass

huber = Huber(y=a, y_pred=pred)
print("The Huber Loss is:", huber)

### Mean Squared Logarifmic Erorr (MSLE)

Данная функция потерь применяется в случаях, когда лучше получать заниженные прогнозы, нежели завышенные.

Важное примечание: из-за присутствия логарифма в формуле **целевая переменная должна быть неотрицательной!**. Единицу добавляем, чтобы случайно не получить логарифм нуля.

$$L_i(a, X_i, Y_i) = (\log(a(x_i)+1) - \log(y_i+1))^2$$

**Задание:** реализуйте MSLE

In [ ]:
def MSLE(y, y_pred):
    #YOUR CODE HERE
    pass

msle = MSLE(y=a, y_pred=pred)
print("The Mean Squared Logarifmic Error is:", msle)

### Quantile Loss

В некоторых задачах штраф за ошибку зависит не только от величины абсолютного отклонения от фактического значения, но и от знака этого отклонения. Например, лучше предсказать спрос больше, чем будет по факту, чем меньше, потому что во втором случае будет потеряна прибыль. В этом случае используется квантильная регрессия со следующей функцией потерь:
$$L_i(y_i, a(x_i)) = \rho_\tau(y_i - x_i^T w),$$
$$\rho_\tau(z) = \begin{cases} \tau z, \quad z > 0, \\ (\tau - 1) z, \quad z \leqslant 0 \end{cases}$$
Параметр $\tau \in (0, 1)$ влияет на то, насколько различаются штрафы за положительную и отрицательную разницу.

Изобразим график квантильной функции потерь вместе с некоторыми другими рассмотренными:

In [ ]:
grid = np.linspace(-3, 3, 100)
quantile_tau = 0.2
mse_loss = grid ** 2
rmse_loss = np.sqrt(mse_loss)
mae_loss = np.abs(grid)
huber_loss = 0.5 * mse_loss * (grid >= -1) * (grid <= 1) + (mae_loss - 0.5) * (grid < -1) + (mae_loss - 0.5)  * (grid > 1)
quantile_loss = quantile_tau * grid * (grid > 0) + (quantile_tau - 1) * grid * (grid <= 0)
plt.plot(grid, mae_loss, label="Absolute Loss")
plt.plot(grid, mse_loss, label="Quadratic Loss")
plt.plot(grid, rmse_loss, label="Root Quadratic Loss")
plt.plot(grid, huber_loss, label="Huber Loss")
plt.plot(grid, quantile_loss, label="Quantile Loss")
plt.xlabel("$y_i - a(x_i)$")
plt.ylabel("$L(y_i, a(x_i))$")
plt.legend()
plt.show()

__Задача.__ Укажите параметр $\tau$, при котором обучение квантильной регрессии равносильно оптимизации MAE.

Проследим наклон прямой в нашей одномерной задаче регрессии при изменении $\tau$:

In [ ]:
plt.scatter(X[7], y)

grid = np.linspace(0, 2, 100)
dat = pd.DataFrame({"x":X[7], "y":y})
mod = smf.quantreg('y ~ x', dat)

for q in np.arange(0.1, 1, 0.1):
    res = mod.fit(q=q)
    plt.plot(grid, grid * res.params["x"] + res.params["Intercept"], linewidth=0.5, label="q = "+str(q))

plt.legend(loc=(1.1, 0.1))
plt.xlabel("x")
plt.ylabel("y")

# Practice

Мы поработаем с данными о сообществах в США. Описание датасета:

http://archive.ics.uci.edu/ml/datasets/communities+and+crime

Датасет на кэггле (в формате .csv):

https://www.kaggle.com/kkanda/communities%20and%20crime%20unnormalized%20data%20set

Будем предсказывать количество насильственных преступлений относительно численности населения.

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import warnings

warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('crimedata.csv', na_values=["?"])

# оставим лишь нужные колонки
requiredColumns = [5, 6] + list(range(11,26)) + list(range(32, 103)) + [145]
data = data[data.columns[requiredColumns]]

# некоторые значения целевой переменной пропущены
X = data.loc[data['ViolentCrimesPerPop'].notnull(), :].drop('ViolentCrimesPerPop', axis=1)
y = data['ViolentCrimesPerPop'][X.index]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### 1 Baseline

Обучим линейную регрессию и выведем качество по метрике MSE на обучающей и тестовой выборке.

In [ ]:
lr = LinearRegression().fit(X_train,y_train)
print ("Train: {}".format(mean_squared_error(y_train, lr.predict(X_train))))
print ("Test: {}".format(mean_squared_error(y_test, lr.predict(X_test))))

# Выведите ниже качество на обучении и тесте, рассчитаное с использованием функции
# MSE(), написанной вами ранее
# Сравните результаты
# YOUR CODE HERE

Популярным решением для регрессионных моделей является **регуляризация**.

Во время оптимизации линейной регрессии, веса при переменных могут получится большими в абсолютных значениях. Это не очень хорошо, поскольку классификатор будет чувствителен к крайне маленьким изменениям в признаках объекта, а значит, переобучен.

Для решения проблемы к функционалу ошибки добавляют так называемсый регуляризатор, который "штрафует" модель за слишком большую норму вектора весов:


$$Q\alpha(w) = Q(w) + \alpha R(w)$$ 

где $R(w)$ - регуляризатор

Наиболее распространенными являются L1 и L2 регуляризаторы
$$L1: R(w) = ||w||_1 = \sum^d_i w_i^2$$

$$L2: R(w) = ||w||_2 = \sum^d_i |w_i|$$

Давайте применим каждый из них к нашей задаче и посмотрим на изменение в результатах.


В качестве метода регуляризации используем Ridge ($L_2$-регуляризация).

In [ ]:
ridge = Ridge(5.0).fit(X_train,y_train)
print ("Train: {}".format(mean_squared_error(y_train, ridge.predict(X_train))))
print ("Test: {}".format(mean_squared_error(y_test, ridge.predict(X_test))))

### 2 Scaling

Попробуем MinMaxScaler.

In [ ]:
sc = MinMaxScaler()
X_train_scaled = pd.DataFrame(data=sc.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(data=sc.transform(X_test), columns=X_test.columns)

**Задание:** Напишите код обучения линейной регресии на масштабированных признаках и выведите ошибку на обучающей и валидационной выборке

In [ ]:
# YOUR CODE HERE

**Задание:** проделайте аналогичную работу, добавив Ridge регуляризацию

In [ ]:
# YOUR CODE HERE

### 3. High/low variance

Полезны ли признаки, имеющие высокую дисперсию? А низкую?

In [ ]:
features_variance = X_train_scaled.var().sort_values(ascending=False)
features_variance.head()

Попробуем удалить признаки с самой низкой дисперсией и посмотреть, как изменится качество. В sklearn есть специальный инструмент для такого наивного отбора признаков. Стоит ли нормализовать перед этим признаки?

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
# можно убрать все признаки, дисперсия которых меньше заданного значения
vs_transformer = VarianceThreshold(0.01)

X_train_var = pd.DataFrame(data=vs_transformer.fit_transform(X_train_scaled), columns=X_train_scaled.columns[vs_transformer.get_support()])
X_test_var = pd.DataFrame(data=vs_transformer.transform(X_test_scaled), columns=X_test_scaled.columns[vs_transformer.get_support()])

X_train_var.shape

In [ ]:
lr = LinearRegression().fit(X_train_var,y_train)
print ("Train: {}".format(mean_squared_error(y_train, lr.predict(X_train_var))))
print ("Test: {}".format(mean_squared_error(y_test, lr.predict(X_test_var))))

In [ ]:
ridge = Ridge(5.0).fit(X_train_var,y_train)
print ("Train: {}".format(mean_squared_error(y_train, ridge.predict(X_train_var))))
print ("Test: {}".format(mean_squared_error(y_test, ridge.predict(X_test_var))))

### 4 Correlation

Можно выбрать k признаков, которые дают наиболее высокие значения корреляции с целевой переменной.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

In [ ]:
# Выбираем 15 лучших признаков
sb = SelectKBest(f_regression, k=15)

X_train_kbest = pd.DataFrame(data=sb.fit_transform(X_train_var, y_train), columns=X_train_var.columns[sb.get_support()])
X_test_kbest = pd.DataFrame(data=sb.transform(X_test_var), columns=X_test_var.columns[sb.get_support()])

In [ ]:
lr = LinearRegression().fit(X_train_kbest,y_train)
print ("Train: {}".format(mean_squared_error(y_train, lr.predict(X_train_kbest))))
print ("Test: {}".format(mean_squared_error(y_test, lr.predict(X_test_kbest))))

In [ ]:
ridge = Ridge(5.0).fit(X_train_kbest,y_train)
print ("Train: {}".format(mean_squared_error(y_train, ridge.predict(X_train_kbest))))
print ("Test: {}".format(mean_squared_error(y_test, ridge.predict(X_test_kbest))))

А можно выбрать самые значимые признаки с точки зрения регрессии с $L_1$-регуляризацией.

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
lasso = Lasso(5.0)
l1_select = SelectFromModel(lasso)

X_train_l1 = pd.DataFrame(data=l1_select.fit_transform(X_train_var, y_train), columns=X_train_var.columns[l1_select.get_support()])
X_test_l1 = pd.DataFrame(data=l1_select.transform(X_test_var), columns=X_test_var.columns[l1_select.get_support()])

X_train_l1.shape

In [ ]:
lr = LinearRegression().fit(X_train_l1,y_train)
print ("Train: {}".format(mean_squared_error(y_train, lr.predict(X_train_l1))))
print ("Test: {}".format(mean_squared_error(y_test, lr.predict(X_test_l1))))

ridge = Ridge(5.0).fit(X_train_l1,y_train)
print ("Train: {}".format(mean_squared_error(y_train, ridge.predict(X_train_l1))))
print ("Test: {}".format(mean_squared_error(y_test, ridge.predict(X_test_l1))))

### 5 Pipeline

А можно сделать все вышеописанное сразу:

In [ ]:
from sklearn.pipeline import Pipeline


pipe = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('variance', VarianceThreshold(0.01)),
    ('selection', SelectFromModel(Lasso(5.0))),
    ('regressor', Ridge(5.0))
])

pipe.fit(X_train, y_train)

pipe.named_steps

In [ ]:
print ("Train: {}".format(mean_squared_error(y_train, pipe.predict(X_train))))
print ("Test: {}".format(mean_squared_error(y_test, pipe.predict(X_test))))

Можно также настраивать параметры с помощью `GridSearch`:

In [ ]:
pipe.get_params()

In [ ]:
param_grid = {
    'variance__threshold': [0.005, 0.0075, 0.009, 0.01, 0.011, 0.012],
    'selection__estimator__alpha': [0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0],
    'regressor__alpha': [0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0]
}
grid_search = GridSearchCV(pipe, param_grid, iid=False, cv=5)

grid_search.fit(X_train, y_train)

In [ ]:
pipe_best = grid_search.best_estimator_
pipe_best.named_steps

In [ ]:
pipe_best.fit(X_train, y_train)
print ("Train: {}".format(mean_squared_error(y_train, pipe_best.predict(X_train))))
print ("Test: {}".format(mean_squared_error(y_test, pipe_best.predict(X_test))))

### Источники
[Лекции Евгения Соколова в рамках курса МО-1](https://github.com/esokolov/ml-course-hse/blob/master/2019-fall/lecture-notes/)

[Семинар Евгения Ковалева в рамках данного курса в 2019 году](https://github.com/KovalevEvgeny/minor2019-iad2/blob/master/sem05_linreg/sem05_linear_regression.ipynb)

[Семинар Евгения Ковалева в рамках данного курса в 2020 году](https://github.com/KovalevEvgeny/minor2020-iad4/blob/master/sem06_linreg/sem06_linreg.ipynb)

[Семинар Евгения Ковалева, посвященный отбору признаков и регуляризации (2020)](https://github.com/KovalevEvgeny/minor2019-iad2/tree/master/sem06_feature_selection_regularization)